In [7]:
# Imports
import logging
import json
import pathlib
from itertools import chain
import numpy
import os
import pandas

from metrics.enumerators import MethodAveragePrecision
from metrics.pascal_voc_evaluator import (
    get_pascalvoc_metrics,
)
from utils import (
    from_file_to_boundingbox,
    get_filenames_of_path,
)

logger: logging.Logger = logging.getLogger(__name__)

# root directory
FOLDER_PATH = pathlib.Path(os.getcwd()).parent.absolute()
MODULE_PATH = FOLDER_PATH.parent.absolute() / "PyTorch-Object-Detection-Faster-RCNN-Tutorial"
DATA_PATH = MODULE_PATH / "pytorch_faster_rcnn_tutorial" / "data" / "heads"


# input and target files
predictions = get_filenames_of_path(DATA_PATH / "predictions")
predictions.sort()
# test-target files
gt_predictions = get_filenames_of_path(DATA_PATH / "test_targets")
gt_predictions.sort()


# get the gt_boxes from disk  || test-targets
gt_boxes = [
    from_file_to_boundingbox(file_name, groundtruth=True)
    for file_name in gt_predictions
]
# reduce list
gt_boxes = list(chain(*gt_boxes))

pred_boxes = [
    from_file_to_boundingbox(file_name, groundtruth=False)
    for file_name in predictions
]
pred_boxes = list(chain(*pred_boxes))

output = get_pascalvoc_metrics(
    gt_boxes=gt_boxes,
    det_boxes=pred_boxes,
    iou_threshold=0.60,
    method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION,
    generate_table=True,
)

per_class, m_ap = output["per_class"], output["m_ap"]
#print(per_class)
head = per_class["Pedestrian"]

In [8]:
for key, value in head.items():
    print(key, type(value))
    if isinstance(value, numpy.ndarray):
        head[key] = value.tolist()
    if isinstance(value, MethodAveragePrecision):
        head[key] = str(value)
    if isinstance(value, pandas.core.frame.DataFrame):
        head[key] = value.to_dict()

with open('pedestrian.json', 'w') as f:
    json.dump(head, f, indent=4)

precision <class 'numpy.ndarray'>
recall <class 'numpy.ndarray'>
AP <class 'numpy.float64'>
interpolated precision <class 'list'>
interpolated recall <class 'list'>
total positives <class 'int'>
total tp <class 'numpy.float64'>
total fp <class 'numpy.float64'>
method <enum 'MethodAveragePrecision'>
iou <class 'float'>
table <class 'pandas.core.frame.DataFrame'>
